In [2]:
import pandas as pd, numpy as np, matplotlib.pyplot as plt
data = pd.read_stata("https://github.com/worldbank/r-econ-visual-library/raw/master/Library/Data/autor-jole-2003.dta")

In [3]:
data

,state,year,ue,cpsths,tempue,hrwage,wkwage,lnhrw,lnwkw,lnftw,...,annemp,totemp,bls_ue,region,statelf,natlf,reglf,natue,regue,union
0,11.0,74.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,361500.0,78714400.0,NaN,1,NaN,0.0,0.0,0.000000,0.000000,NaN
1,11.0,75.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,356900.0,77387696.0,NaN,1,NaN,0.0,0.0,0.000000,0.000000,20.700001
2,11.0,76.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,375300.0,79863504.0,NaN,1,NaN,0.0,0.0,0.000000,0.000000,NaN
3,11.0,77.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,387800.0,82970000.0,NaN,1,NaN,0.0,0.0,0.000000,0.000000,NaN
4,11.0,78.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,405600.0,87369200.0,0.062,1,432409.3750,92354648.0,5544384.5,0.060438,0.057785,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1195,95.0,93.0,0.043933,0.003606,0.000000,13.194592,520.331238,2.437165,6.041272,6.222130,...,538800.0,111060304.0,0.043,9,563009.3750,118561184.0,17981786.0,0.068920,0.088055,NaN
1196,95.0,94.0,0.060942,0.003455,0.000000,13.323984,520.674988,2.436631,6.012619,6.188493,...,536200.0,114176896.0,0.061,9,571033.0000,120860816.0,18058438.0,0.060753,0.079533,NaN
1197,95.0,95.0,0.057788,0.002229,0.000000,13.050425,507.867493,2.417453,5.979689,6.177753,...,532800.0,117092096.0,0.059,9,566206.1875,123315104.0,18319228.0,0.055675,0.072979,NaN
1198,95.0,96.0,0.058244,0.004537,0.001337,13.339598,509.207947,2.430573,5.967892,6.180829,...,530700.0,119550296.0,0.064,9,566987.1875,125680008.0,18735586.0,0.053730,0.069797,NaN
